# **1. CONFIGURATION**
install path, output path, default settings

In [ ]:
#mobile optimization require updates from https://github.com/viyiviyi/stable-diffusion-webui.git, unreliable
#mobileOptimize=True 

#install path on /kaggle have more storage for models. ( 70 GB ++ )
#install path on /kaggle/working have the ability for quick run after the first run ( using persistency files in notebook options)  
install_path="/kaggle" 

# Output path is separate from install folder so you can use persistency to save image output
# enable persistence in notebook option so you can load your image the nexttime it runs
output_path="/kaggle/working"

linkHypernetworksDir=False # Link the hypernetworks directory to the output directory. It is recommended to enable it if you need to train hypernetworks
linkEmbeddingsDir=False # Link the embeddings directory to the output directory. It is recommended to enable it if you need to train embeddings
linkTextual_inversionDir=False # Link the textual_inversion directory to the output directory. It is recommended to enable it if you need to save the output files of the training process.

# To skip downloading controlnet model, set to False (useful if you have input files linked or just want to skip)
dl_controlnet=True

# original webui_setting https://github.com/2575044704/sd-configs4.git
# this sample use mine
webui_settings = 'https://github.com/emjayoh/sd-configs'
setting_file = '/kaggle/input/configs/config.json'
ui_config_file = '/kaggle/input/configs/ui-config.json'

from pathlib import Path
import os
import time
import re

os.environ['install_path'] = install_path
os.environ['output_path'] = output_path
os.environ['webui_settings'] = webui_settings

print('Done')

# **2. INSTALL WEBUI & DEPENDENCIES**
Nothing to do here, just run it.

In [ ]:
#INSTALL & DEPENDENCIES
# 🐣 Please follow me for new updates https://twitter.com/camenduru
# 🔥 Please join our discord server https://discord.gg/k5BwmmvJJU
def link_dir():
#  Link output directory Because SD is deployed in the stable-diffusion-webui directory, only the outputs directory is placed in the output_path/ directory for easy download after running
    !mkdir -p $output_path/outputs
    !rm -rf $install_path/stable-diffusion-webui/outputs
    !ln -s -r $output_path/outputs $install_path/stable-diffusion-webui/ # output directory
    !mkdir -p $output_path/log
    !rm -rf $install_path/stable-diffusion-webui/log
    !ln -s -r $output_path/log $install_path/stable-diffusion-webui/
# Link the hypernetworks directory
    if linkHypernetworksDir:
        !rm -rf $install_path/stable-diffusion-webui/models/hypernetworks
        !mkdir -p $output_path/hypernetworks
        !ln -s -r $output_path/hypernetworks $install_path/stable-diffusion-webui/models/
# Link to the embeddings directory
    if linkEmbeddingsDir:
        !rm -rf $install_path/stable-diffusion-webui/embeddings
        !mkdir -p $output_path/embeddings
        !ln -s -r $output_path/embeddings $install_path/stable-diffusion-webui/
# Link the training output directory Folder links will cause the function to be unavailable
    if linkTextual_inversionDir:
        !rm -rf $install_path/stable-diffusion-webui/textual_inversion
        !mkdir -p $output_path/textual_inversion/
        !ln -s -r $output_path/textual_inversion $install_path/stable-diffusion-webui/
        
def use_config():
    %cd $install_path/stable-diffusion-webui
    !mkdir -p tmp
    %cd tmp
    !git clone $webui_settings sd-configs
    !cp -rf sd-configs/dist/* $install_path/stable-diffusion-webui
    
def start():
    %cd $install_path/stable-diffusion-webui
    #write arguments below
    !python launch.py --listen --no-half-vae --api --xformers --enable-insecure-extension-access --theme dark --gradio-queue 
    #useful arguments: --lowvram --medvram    
def main():
    use_config()
    link_dir()
    start()
    
%cd $install_path
!apt -y update -qq
!apt -y install -qq aria2
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1-f https://download.pytorch.org/whl/torch_stable.html
!pip install https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
!pip install huggingface-hub==0.11.0 -U
!pip install -q xformers==0.0.20 -U
!pip install gradio==3.39.0
    #STABLE DIFFUSION BELOW

!git clone https://github.com/camenduru/stable-diffusion-webui #$install_path/stable-diffusion-webui
# !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-nsfw-censor $install_path/stable-diffusion-webui/extensions/stable-diffusion-webui-nsfw-censor #kaggle safety features to prevent abuse and TOS infringemenet. Replace NSFW images with blank.

stable_diffusion_path = os.path.join(install_path, "stable-diffusion-webui")
# Get the list of files and directories inside stable-diffusion
files_sd = os.listdir(stable_diffusion_path)
# Print the list of files and directories
print(files_sd)

!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O $install_path/stable-diffusion-webui/scripts/run_n_times.py

%cd $install_path
#extra NSFW FILTER EXTENSIONS TO PREVENT ABUSE    
# !git clone https://github.com/jovijovi/stable-diffusion-webui-nsfw-filter $install_path/stable-diffusion-webui/extensions/stable-diffusion-webui-nsfw-filter

# CONTROL NET BELOW v1
if dl_controlnet:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d $install_path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
    
!npm install -g localtunnel
import subprocess
import threading
import time
import socket
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()
print('DONE.')

# **3. EXTENSION TO DOWNLOAD**
Just put the link like these. 
Enable update by using `ExtUpdate=True` if you use persistency (`install_path` on `/kaggle` ) or during run. Actually just run this cell, it's a gimmick and will install or update accordingly. 

In [ ]:
#put the link for extensions download or use # to skip
ExtUpdate=False #set to true before run to update

extensionsdl=[
  'https://github.com/deforum-art/deforum-for-automatic1111-webui', #from original, video generation
  'https://github.com/camenduru/stable-diffusion-webui-huggingface', #from original, to save model withing hugginface
  'https://github.com/kohya-ss/sd-webui-additional-networks', #from original, alternative to use LoRA
  'https://github.com/Mikubill/sd-webui-controlnet', #from original, very amazing control feature
  'https://github.com/camenduru/openpose-editor', #from original, use with controlnet
  'https://github.com/nonnonstop/sd-webui-3d-open-pose-editor', #3d version of openpose
  'https://github.com/jexom/sd-webui-depth-lib', #from original, idk
  'https://github.com/hnmr293/posex', #from original, idk
  'https://github.com/etherealxx/batchlinks-webui', #from original, to get multiple model
  'https://github.com/camenduru/stable-diffusion-webui-catppuccin', #from original, colorful theme
  'https://github.com/KohakuBlueleaf/a1111-sd-webui-locon', #from original, to load lycoris and locon as lora
  'https://github.com/camenduru/sd-civitai-browser', #easy retrieve model & lora from civitai
  'https://github.com/DominikDoom/a1111-sd-webui-tagcomplete', #tag suggestion in prompt booru style
  'https://github.com/zanllp/sd-webui-infinite-image-browsing' #better image browsing
  #'https://github.com/camenduru/sd_webui_stealth_pnginfo', #from original, not necessary since discord didn't strip png
  #'https://github.com/Interpause/auto-sd-paint-ext', #krita plugin, need --api
  #'https://github.com/AbdullahAlfaraj/Auto-Photoshop-StableDiffusion-Plugin' #photoshop plugin, need --api
]


# below is the loop code to download extensions
for ext_url in extensionsdl:
    ext_name = ext_url.split("/")[-1]
    ext_path = os.path.join(install_path, "stable-diffusion-webui", "extensions", ext_name)

    if ExtUpdate:
        if os.path.isdir(ext_path):
            print(f"Updating {ext_name}")
            os.system(f"git -C {ext_path} reset --hard")
            os.system(f"git -C {ext_path} pull")
        else:
            print(f"Extension {ext_name} is not installed, cannot update. Installing")
            os.system(f'git clone {ext_url} {ext_path}')        
    else:
        if not os.path.isdir(ext_path):
            print(f"Installing {ext_name}")
            os.system(f'git clone {ext_url} {ext_path}')
        else:
            print(f"Extension {ext_name} is already installed. Updating")
            os.system(f"git -C {ext_path} reset --hard")
            os.system(f"git -C {ext_path} pull")
            
# Image browser fix
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser $install_path/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
%cd $install_path/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git checkout 1d5c2e7
            
print('Done')

# **4. MODEL TO DOWNLOAD**
Download Models from direct link only. 
Use CivitAI extension for CivitAI in webui.
Use Etherealxx batchlink for others.

In [ ]:
# only support direct link, use batchlink or civitai browser inside webui for more, or use input data
# "model_name.extension": "model_download_link", 
# you can add # in front to disable without deleting the codes
modelsdl = {
    #"anything-v4.0-pruned.ckpt": "https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0-pruned.ckpt",
#     "anything-v4.5-pruned.ckpt": "https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt",
#     "anything-v4.5-pruned.vae.pt": "https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0.vae.pt",
   "dreamshaperXL.safetensors": "https://civitai.com/api/download/models/126688",
    "mbbxl.safetensors": "https://civitai.com/api/download/models/127166",
    #"SukiyakiMix-v1.0-fp32.safetensors": "https://huggingface.co/Vsukiyaki/SukiyakiMix-v1.0/resolve/main/SukiyakiMix-v1.0-fp32.safetensors",
    
    #"mdjrny-v4.ckpt": "https://huggingface.co/ckpt/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt",
    
    #"dreamlike-photoreal-2.0.safetensors": "https://huggingface.co/ckpt/dreamlike-photoreal-2.0/resolve/main/dreamlike-photoreal-2.0.safetensors",
    
    #"OpenAnimeJourney.ckpt": "https://huggingface.co/ckpt/OpenAnimeJourney/resolve/main/OpenAnimeJourney.ckpt",
    #"Anything-V3.0.vae.pt": "https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt",
    
    #"AOM3.safetensors": "https://huggingface.co/ckpt/OrangeMixs/resolve/main/AOM3.safetensors",
    #"AOM3A1.safetensors": "https://huggingface.co/ckpt/OrangeMixs/resolve/main/AOM3A1.safetensors",
    #"AOM3A2.safetensors": "https://huggingface.co/ckpt/OrangeMixs/resolve/main/AOM3A2.safetensors",
    #"AOM3A3.safetensors": "https://huggingface.co/ckpt/OrangeMixs/resolve/main/AOM3A3.safetensors",
    #"orangemix.vae.pt": "https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt",
}

# below is the loop code to download models
modelsdl_aria = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"

for modelname, modelurl in modelsdl.items():
    print (f"Downloading {modelname}")
    download_cmd = f"{modelsdl_aria} {modelurl} -d {install_path}/stable-diffusion-webui/models/Stable-diffusion -o {modelname}"
    os.system(download_cmd)
    print (f"{modelname} downloaded")
# "model_name.extension": "model_download_link", "model_name.extension": "model_download_link"
# you can add # in front to disable without deleting the codes

print('Done. Start main cell to run WEBUI')

In [ ]:
# WEBUI MAIN CELL
#changing requirements to fits kaggle environment
%cd $install_path/stable-diffusion-webui
!sed -i -e 's/numpy==1.23.3/numpy==1.21.6/g' requirements_versions.txt
!sed -i -e 's/blendmodes==2022/blendmodes==2021/g' requirements_versions.txt
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' requirements_versions.txt
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' $install_path/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' $install_path/stable-diffusion-webui/launch.py

reloadwebui=False  #set to True for fast reloading this cell
if reloadwebui:
        %cd $install_path/stable-diffusion-webui
        with open("launch.py") as f:
            if "skip_install = False" in f.read():
                print("Setting skip_install to True in launch.py")
                !sed -i "s/skip_install = False/skip_install = True/" launch.py 
        print("Running launch.py. From there, it's stable-diffusion-webui's code.")
        
        !python launch.py

print('Loading WEBUI. Wait until "your url is" and "Running on public URL" are given.')
main()
print('WEBUI stopped. Re run this cell for restart or head to next cell to create downloadable output.zip')

In [ ]:
# FOR DOWNLOADING OUTPUTS AS ZIP, set as True
zipoutput=False #set to True before running to use
if zipoutput:
    !zip -r /kaggle/working/outputs.zip $output_path/outputs #zip all outputs folder
    #!zip -r /kaggle/working/outputs.zip $output_path/outputs/txt2img-images/yyyy-mm-dd #for text2image specific date
    #!zip -r /kaggle/working/outputs.zip $output_path/outputs/txt2img-grids/yyyy-mm-dd #for t2i grid specific date
    #!zip -r /kaggle/working/outputs.zip $output_path/outputs/img2img-images/yyyy-mm-dd #for image2image specific date
    #!zip -r /kaggle/working/outputs.zip $output_path/outputs/img2img-grids/yyyy-mm-dd #for i2i grid specific date
    print('DONE. Refresh Output to download outputs.zip')


In [ ]:
# FOR COPY FRESH CONFIG FILES FOR POSSIBLY EXTRA CONFIGS FROM NEW EXTENSION, set freshconfig  as  true
# RUN WEBUI,STOP WEBUI CELL AFTER RUNNING, THEN RUN THIS CELL
freshconfig=False #set to True before run to use
if freshconfig:
    os.makedirs('/kaggle/working/configs')
    import shutil
    src_path = r"/kaggle/stable-diffusion-webui/config.json"
    dst_path = r"/kaggle/working/configs/config.json"
    shutil.copy(src_path, dst_path)
    import shutil
    src_path = r"/kaggle/stable-diffusion-webui/ui-config.json"
    dst_path = r"/kaggle/working/configs/ui-config.json"
    shutil.copy(src_path, dst_path)
    print('Copied')
    !zip -r /kaggle/working/configs.zip /kaggle/working/configs
    print('DONE. Refresh Output to download configs.zip by clicking three vertical dots>Download')

Big thanks to [Camenduru](https://twitter.com/camenduru) 💌 and all [Contributor/Collaborator](https://discord.gg/k5BwmmvJJU) for always up to date and reliable colab.

Thanks to [ChatGPT](https://chat.openai.com/chat) for providing 24/7 assistance. You can look how dependent i'm in [using ChatGPT](https://sharegpt.com/c/Z8KX0c4). 

Special thanks ✌ [machievel23](https://github.com/machiavel23), [etherealxx](https://github.com/etherealxx), [ddPn08](https://github.com/ddpn08/) and [viyiviyi](https://github.com/viyiviyi) for their inspiring notebook.